In [1]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2014 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2014 PUMS data

In [2]:
variable_types = {"RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","TAXP","SMP","CONP","TEN","VACS","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2014_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,8513,139.00,02,5.00,,,00700,,3.00,nan,,0700,056,000015000,1.00,nan
1,H,6708,177.00,03,2.00,,,,,nan,7.00,,,,,nan,nan
2,H,7301,259.00,04,2.00,,,02300,,3.00,nan,,2451,040,000073000,2.00,nan
3,H,1903,165.00,03,2.00,,01100,,,1.00,nan,0215000,,,000086000,3.00,52.00
4,H,6509,174.00,03,2.00,,,00650,,3.00,nan,,0990,018,000064500,3.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,04,2.00,,01000,,,1.00,nan,0180000,,,000128000,6.00,24.00
154320,H,6515,178.00,02,2.00,,,01000,,nan,1.00,,,,,nan,nan
154321,H,1902,165.00,02,6.00,,,00800,,3.00,nan,,0900,027,000040000,1.00,nan
154322,H,9904,166.00,04,2.00,,01700,,00080,1.00,nan,0375000,,,000113430,4.00,58.00


In [3]:
pums_df["BDSP"]=pd.to_numeric(pums_df["BDSP"], errors='coerce')
pums_df["RNTP"]=pd.to_numeric(pums_df["RNTP"], errors='coerce')
pums_df["CONP"]=pd.to_numeric(pums_df["CONP"], errors='coerce')
pums_df["MRGP"]=pd.to_numeric(pums_df["MRGP"], errors='coerce')
pums_df["SMP"]=pd.to_numeric(pums_df["SMP"], errors='coerce')

In [4]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,4.00,2.00,nan,1000.00,nan,nan,1.00,nan,0180000,,,000128000,6.00,24.00
154320,H,6515,178.00,2.00,2.00,nan,nan,1000.00,nan,nan,1.00,,,,,nan,nan
154321,H,1902,165.00,2.00,6.00,nan,nan,800.00,nan,3.00,nan,,0900,027,000040000,1.00,nan
154322,H,9904,166.00,4.00,2.00,nan,1700.00,nan,80.00,1.00,nan,0375000,,,000113430,4.00,58.00


## Part A2. Assign county id to each puma

In [5]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,4.00,2.00,nan,1000.00,nan,nan,1.00,nan,0180000,,,000128000,6.00,24.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
154320,H,6515,178.00,2.00,2.00,nan,nan,1000.00,nan,nan,1.00,,,,,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
154321,H,1902,165.00,2.00,6.00,nan,nan,800.00,nan,3.00,nan,,0900,027,000040000,1.00,nan,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN
154322,H,9904,166.00,4.00,2.00,nan,1700.00,nan,80.00,1.00,nan,0375000,,,000113430,4.00,58.00,Stanislaus CA,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,4.00,2.00,nan,1000.00,nan,nan,1.00,nan,0180000,,,000128000,6.00,24.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
154320,H,6515,178.00,2.00,2.00,nan,nan,1000.00,nan,nan,1.00,,,,,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
154321,H,1902,165.00,2.00,6.00,nan,nan,800.00,nan,3.00,nan,,0900,027,000040000,1.00,nan,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno
154322,H,9904,166.00,4.00,2.00,nan,1700.00,nan,80.00,1.00,nan,0375000,,,000113430,4.00,58.00,Stanislaus CA,NaN,NaN,NaN,NaN,NaN,NaN,Stanislaus


## Part A3. Bring in and clean, reformat county AMIs data

In [8]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2014_amis.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,19650,22450,28050,32550,37050
1,Alameda County\n4-Per,Very Low Income,32750,37400,46750,54250,61750
2,Alameda County\n4-Per,Low Income,47350,54100,67600,78450,89250
3,Alameda County\n4-Per,Median Income,65450,74800,93500,108450,123400
4,Alameda County\n4-Per,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12500,14250,17800,20650,23500
286,Yuba County\n4-Person,Very Low Income,20800,23800,29700,34500,39250
287,Yuba County\n4-Person,Low Income,33250,38000,47500,55100,62700
288,Yuba County\n4-Person,Median Income,41600,47500,59400,68900,78400


In [9]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,19650,22450,28050,32550,37050
1,Alameda,Very Low Income,32750,37400,46750,54250,61750
2,Alameda,Low Income,47350,54100,67600,78450,89250
3,Alameda,Median Income,65450,74800,93500,108450,123400
4,Alameda,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12500,14250,17800,20650,23500
286,Yuba,Very Low Income,20800,23800,29700,34500,39250
287,Yuba,Low Income,33250,38000,47500,55100,62700
288,Yuba,Median Income,41600,47500,59400,68900,78400


In [10]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,491.25,561.25,701.25,813.75,926.25
5,Alpine,428.75,490.00,612.50,711.25,808.75
10,Amador,380.00,435.00,542.50,630.00,716.25
15,Butte,308.75,352.50,440.00,511.25,581.25
20,Calaveras,367.50,420.00,525.00,610.00,693.75
25,Colusa,303.75,347.50,433.75,503.75,573.75
30,Contra Costa,491.25,561.25,701.25,813.75,926.25
35,Del Norte,303.75,347.50,433.75,503.75,573.75
40,El Dorado,400.00,457.50,571.25,663.75,755.00
45,Fresno,303.75,347.50,433.75,503.75,573.75


In [11]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,818.75,935.00,1168.75,1356.25,1543.75
6,Alpine,715.00,816.25,1020.00,1183.75,1347.50
11,Amador,633.75,723.75,903.75,1048.75,1193.75
16,Butte,513.75,587.50,733.75,851.25,968.75
21,Calaveras,612.50,700.00,875.00,1015.00,1155.00
26,Colusa,507.50,580.00,723.75,840.00,956.25
31,Contra Costa,818.75,935.00,1168.75,1356.25,1543.75
36,Del Norte,507.50,580.00,723.75,840.00,956.25
41,El Dorado,666.25,761.25,951.25,1103.75,1256.25
46,Fresno,507.50,580.00,723.75,840.00,956.25


In [12]:
li_df = df[df.Income_Category =="Low Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1183.75,1352.50,1690.00,1961.25,2231.25
7,Alpine,1118.75,1278.75,1597.50,1853.75,2108.75
12,Amador,1012.50,1157.50,1446.25,1678.75,1910.00
17,Butte,822.50,940.00,1173.75,1362.50,1550.00
22,Calaveras,980.00,1120.00,1400.00,1625.00,1848.75
27,Colusa,811.25,926.25,1157.50,1343.75,1528.75
32,Contra Costa,1183.75,1352.50,1690.00,1961.25,2231.25
37,Del Norte,811.25,926.25,1157.50,1343.75,1528.75
42,El Dorado,1066.25,1218.75,1522.50,1766.25,2010.00
47,Fresno,811.25,926.25,1157.50,1343.75,1528.75


In [13]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1636.25,1870.00,2337.50,2711.25,3085.00
8,Alpine,1487.50,1700.00,2125.00,2465.00,2805.00
13,Amador,1265.00,1446.25,1807.50,2096.25,2386.25
18,Butte,1027.50,1173.75,1467.50,1702.50,1937.50
23,Calaveras,1225.00,1400.00,1750.00,2030.00,2310.00
28,Colusa,1013.75,1157.50,1447.50,1678.75,1911.25
33,Contra Costa,1636.25,1870.00,2337.50,2711.25,3085.00
38,Del Norte,1013.75,1157.50,1447.50,1678.75,1911.25
43,El Dorado,1331.25,1522.50,1902.50,2207.50,2511.25
48,Fresno,1013.75,1157.50,1447.50,1678.75,1911.25


In [14]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,1963.75,2243.75,2805.00,3253.75,3702.50
9,Alpine,1785.00,2040.00,2550.00,2957.50,3366.25
14,Amador,1517.50,1735.00,2168.75,2516.25,2862.50
19,Butte,1232.50,1408.75,1761.25,2042.50,2325.00
24,Calaveras,1470.00,1680.00,2100.00,2436.25,2772.50
29,Colusa,1216.25,1390.00,1737.50,2015.00,2293.75
34,Contra Costa,1963.75,2243.75,2805.00,3253.75,3702.50
39,Del Norte,1216.25,1390.00,1737.50,2015.00,2293.75
44,El Dorado,1597.50,1826.25,2282.50,2647.50,3012.50
49,Fresno,1216.25,1390.00,1737.50,2015.00,2293.75


In [15]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1183.75,1352.50,1690.00,1961.25,2231.25,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
1,Alpine,428.75,490.00,612.50,711.25,808.75,715.00,816.25,1020.00,1183.75,1347.50,1118.75,1278.75,1597.50,1853.75,2108.75,1487.50,1700.00,2125.00,2465.00,2805.00,1785.00,2040.00,2550.00,2957.50,3366.25
2,Amador,380.00,435.00,542.50,630.00,716.25,633.75,723.75,903.75,1048.75,1193.75,1012.50,1157.50,1446.25,1678.75,1910.00,1265.00,1446.25,1807.50,2096.25,2386.25,1517.50,1735.00,2168.75,2516.25,2862.50
3,Butte,308.75,352.50,440.00,511.25,581.25,513.75,587.50,733.75,851.25,968.75,822.50,940.00,1173.75,1362.50,1550.00,1027.50,1173.75,1467.50,1702.50,1937.50,1232.50,1408.75,1761.25,2042.50,2325.00
4,Calaveras,367.50,420.00,525.00,610.00,693.75,612.50,700.00,875.00,1015.00,1155.00,980.00,1120.00,1400.00,1625.00,1848.75,1225.00,1400.00,1750.00,2030.00,2310.00,1470.00,1680.00,2100.00,2436.25,2772.50
5,Colusa,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
6,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1183.75,1352.50,1690.00,1961.25,2231.25,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
7,Del Norte,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
8,El Dorado,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
9,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75


## Part A4. Add count dummies for each unit based on income and unit type

In [16]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,4.00,2.00,nan,1000.00,nan,nan,1.00,nan,0180000,,,000128000,6.00,24.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75
154320,H,6515,178.00,2.00,2.00,nan,nan,1000.00,nan,nan,1.00,,,,,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75
154321,H,1902,165.00,2.00,6.00,nan,nan,800.00,nan,3.00,nan,,0900,027,000040000,1.00,nan,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
154322,H,9904,166.00,4.00,2.00,nan,1700.00,nan,80.00,1.00,nan,0375000,,,000113430,4.00,58.00,Stanislaus CA,NaN,NaN,NaN,NaN,NaN,NaN,Stanislaus,326.25,372.50,465.00,540.00,615.00,542.50,620.00,775.00,900.00,1023.75,868.75,992.50,1240.00,1438.75,1637.50,1085.00,1240.00,1550.00,1797.50,2046.25,1302.50,1487.50,1860.00,2157.50,2455.00


In [17]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,4.00,2.00,nan,1000.00,nan,nan,1.00,nan,0180000,,,000128000,6.00,24.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
154320,H,6515,178.00,2.00,2.00,nan,nan,1000.00,nan,nan,1.00,,,,,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [18]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,nan
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,H,6508,83.00,4.00,2.00,nan,1000.00,nan,nan,1.00,nan,0180000,,,000128000,6.00,24.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
154320,H,6515,178.00,2.00,2.00,nan,nan,1000.00,nan,nan,1.00,,,,,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00

# Part B2. Add count dummies for each unit based on income and unit type

In [19]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75,...,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154319,

# Part C1. Multiply each dummy by the the household weight variable

In [20]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0.00,139.00,0.00,0.00,0.00,0.00,139.00,0.00,0.00,0.00,0.00,139.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,259.00,0.00,0.00,0.00,0.00,259.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,113

In [21]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
5153,H,3705,69.00,4.00,2.00,nan,nan,2700.00,nan,3.00,nan,,2908,019,000187400,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,69.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5902,H,3720,59.00,4.00,2.00,nan,nan,3000.00,nan,3.00,nan,,3337,022,000185000,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,59.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6530,H,3760,70.00,4.00,2.00,nan,nan,3000.00,nan,3.00,nan,,3257,009,000425000,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,70.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6988,H,7901,101.00,4.00,2.00,nan,nan,3700.00,nan,3.00,nan,,3980,101,000011000,1.00,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,1507.50,1748.75,1990.00,1347.50,1540.00,1925.00,2232.50,2541.25,1617.50,1847.50,2310.00,2680.00,3048.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,101.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7168,H,7301,59.00,4.00,2.00,nan,nan,3700.00,nan,3.00,nan,,3981,055,000087300,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,11

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [22]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0.00,0.00,0.00,139.00,0.00,0.00,0.00,0.00,139.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
1,H,6708,177.00,3.00,2.00,nan,nan,nan,nan,nan,7.00,,,,,nan,nan,Sacramento CA,NaN,NaN,NaN,NaN,NaN,NaN,Sacramento,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,H,7301,259.00,4.00,2.00,nan,nan,2300.00,nan,3.00,nan,,2451,040,000073000,2.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,0.00,259.00,0.00,0.00,0.00,0.00,259.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,1903,165.00,3.00,2.00,nan,1100.00,nan,nan,1.00,nan,0215000,,,000086000,3.00,52.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,6509,174.00,3.00,2.00,nan,nan,650.00,nan,3.00,nan,,0990,018,000064500,3.00,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,15

In [23]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,8513,139.00,2.00,5.00,nan,nan,700.00,nan,3.00,nan,,0700,056,000015000,1.00,nan,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0.00,0.00,0.00,139.00,0.00,0.00,0.00,0.00,139.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
27,H,7313,59.00,1.00,9.00,nan,nan,550.00,nan,3.00,nan,,0550,015,000044600,1.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,59.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
33,H,1905,81.00,3.00,7.00,nan,nan,750.00,nan,3.00,nan,,0850,040,000025400,1.00,nan,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,...,0.00,0.00,0.00,0.00,81.00,0.00,0.00,0.00,0.00,81.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
34,H,3719,178.00,1.00,5.00,nan,nan,1100.00,nan,3.00,nan,,1233,044,000033300,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,178.00,0.00,0.00,0.00,0.00,178.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
40,H,5903,166.00,1.00,5.00,nan,nan,1400.00,nan,3.00,nan,,1440,010,000170000,1.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,953.75,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,25

# Part D2. Aggregating at the PUMA level

In [24]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,363.00,569.00,674.00,458.00,205.00,1568.00,1731.00,2666.00,1068.00,603.00,2730.00,5368.00,7983.00,2212.00,672.00,3482.00,7809.00,11922.00,3278.00,781.00,3482.00,8367.00,12430.00,3564.00,869.00,0.00,65.00,413.00,422.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
102,965.00,3468.00,2836.00,1312.00,121.00,3866.00,9690.00,8040.00,2887.00,121.00,5929.00,18416.00,13102.00,5209.00,658.00,6764.00,22996.00,16243.00,6007.00,737.00,6911.00,23828.00,17353.00,6198.00,737.00,66.00,157.00,62.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,67.00,0.00,0.00,0.00,0.00,132.00,268.00,0.00,0.00,0.00,198.00,268.00,51.00,0.00,0.00,0.00,165.00,0.00,0.00
103,640.00,778.00,419.00,285.00,0.00,752.00,2066.00,1518.00,1079.00,62.00,1312.00,6230.00,3788.00,1885.00,271.00,2123.00,7609.00,5092.00,2194.00,338.00,2123.00,7775.00,5468.00,2678.00,405.00,0.00,134.00,262.00,359.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,73.00,0.00,0.00,0.00,0.00,141.00,0.00,0.00,0.00,0.00,141.00,171.00,0.00,0.00,0.00,141.00,171.00,0.00,0.00,0.00,0.00,0.00,0.00
104,0.00,995.00,2264.00,1735.00,542.00,570.00,3446.00,7403.00,3620.00,957.00,718.00,4019.00,10090.00,6391.00,1705.00,935.00,4140.00,10544.00,6682.00,1705.00,1007.00,4260.00,10544.00,6682.00,1705.00,93.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,166.00,0.00,0.00,0.00,0.00,166.00,0.00,0.00,0.00,49.00,166.00,0.00,0.00,0.00,0.00,0.00,0.00
105,72.00,1094.00,515.00,622.00,461.00,297.00,2397.00,2545.00,1232.00,715.00,916.00,7870.00,10648.00,3010.00,928.00,993.00,10106.00,13697.00,5179.00,1196.00,993.00,10451.00,13899.00,5179.00,1346.00,214.00,73.00,0.00,81.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,68.00,51.00,239.00,0.00,0.00,68.00,603.00,239.00,0.00,0.00,68.00,603.00,239.00,0.00,0.00,68.00,603.00,239.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,63.00,1015.00,882.00,787.00,41.00,131.00,2208.00,2568.00,1567.00,399.00,476.00,5677.00,9537.00,5385.00,1297.00,476.00,5819.00,10202.00,5810.00,1634.00,476.00,5819.00,10202.00,6217.00,1857.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,137.00,0.00,0.00,0.00,0.00,137.00,0.00,0.00,0.00,0.00,228.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,422.00,847.00,707.00,135.00,0.00,1611.00,2307.00,1711.00,178.00,439.00,4386.00,7206.00,4043.00,1496.00,651.00,5120.00,8098.00,4896.00,1573.00,651.00,5256.00,8098.00,4979.00,1573.00,277.00,0.00,0.00,0.00,0

In [25]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [26]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,58.00,169.00,69.00,87.00,126.00,250.00,479.00,575.00,136.00,337.00,491.00,1486.00,1708.00,395.00,406.00,1125.00,2421.00,3826.00,460.00,406.00,1125.00,2874.00,3974.00,607.00,406.00,0.00,65.00,253.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
102,138.00,109.00,426.00,95.00,0.00,1171.00,1626.00,990.00,300.00,0.00,1712.00,3304.00,1845.00,791.00,130.00,2072.00,4655.00,2448.00,957.00,130.00,2072.00,5023.00,3254.00,957.00,130.00,66.00,157.00,62.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,74.00,0.00,0.00,0.00,0.00,74.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
103,0.00,0.00,0.00,149.00,0.00,0.00,213.00,258.00,287.00,62.00,366.00,876.00,1147.00,287.00,126.00,430.00,1386.00,1517.00,287.00,126.00,430.00,1386.00,1782.00,287.00,126.00,0.00,134.00,79.00,141.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
104,0.00,69.00,175.00,289.00,0.00,201.00,827.00,1624.00,565.00,0.00,349.00,898.00,2394.00,1168.00,587.00,349.00,1019.00,2443.00,1459.00,587.00,421.00,1019.00,2443.00,1459.00,587.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,0.00,370.00,62.00,323.00,0.00,0.00,514.00,535.00,323.00,0.00,0.00,1808.00,3454.00,370.00,0.00,77.00,3139.00,4711.00,1013.00,60.00,77.00,3251.00,4835.00,1013.00,60.00,0.00,0.00,0.00,81.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.00,149.00,0.00,0.00,0.00,68.00,406.00,133.00,187.00,0.00,183.00,1424.00,1978.00,803.00,0.00,183.00,1566.00,2199.00,914.00,0.00,183.00,1566.00,2199.00,1223.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,0.00,97.00,0.00,105.00,0.00,0.00,321.00,62.00,105.00,0.00,106.00,1435.00,1690.00,287.00,284.00,318.00,1885.00,2362.00,681.00,361.00,318.00,1989.00,2362.00,681.00,361.00,277.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11105,0.00,36.00,0.00,66.00,0.00,0.00,645.00,

In [27]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")